In [460]:
#enable multiple outputs per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import datetime as dt
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
import re
from collections import Counter

from newsapi import NewsApiClient


newsapi = NewsApiClient(api_key="a1c9a16f08974282a0e45bed8577103b")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Christophe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Christophe\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Christophe\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [461]:
#getting news articles as dataframe from a reliable news source
def get_news_reliable(keyword):
    articles = newsapi.get_everything(
        q=keyword, 
        sources='cnn, cbs-news, bbc-news',
        domains='http://www.cnn.com, http://www.cbsnews.com, http://www.bbc.com',
        from_param='2020-01-01',
        to='2020-01-28',
        language='en',
        sort_by='popularity',
        page_size=100)
    articles = pd.DataFrame(articles)
    articles = pd.concat([articles.drop(['articles'], axis=1), articles['articles'].apply(pd.Series)], axis=1)
    return articles

In [462]:
#getting news articles as dataframe from an unreliable news source
def get_news_unreliable(keyword):
    articles = newsapi.get_everything(
        q=keyword, 
        sources='the-verge, fox-news',
        domains='http://www.theverge.com, http://www.foxnews.com',
        from_param='2020-01-01',
        to='2020-01-28',
        language='en',
        sort_by='popularity',
        page_size=100)
    articles = pd.DataFrame(articles)
    articles = pd.concat([articles.drop(['articles'], axis=1), articles['articles'].apply(pd.Series)], axis=1)
    return articles

def get_dataFrame(articles):
    article_results =[]
    for i in range(len(file)):
        article_dict={}
        article_dict['title'] = file[i]['title']
        article_dict['author'] = file[i]['author']
        article_dict['source'] = file[i]['source']
        article_dict['description'] = file[i]['description']
        article_dict['content'] = file[i]['content']
        article_dict['pub_date'] = file[i]['publishedAt']
        article_dict['url'] = file[i]['url']
        article_dict['photo_url'] = file[i]['urlToImage']
        
        article_results.append(article_dict)
    return article_results

In [463]:
news_reliable = get_news_reliable('coronavirus')
news_unreliable = get_news_unreliable('coronavirus')

In [464]:
news_reliable.head(n=1)
news_unreliable.head(n=1)

,status,totalResults,source,author,title,description,url,urlToImage,publishedAt,content
0,ok,293,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,New China virus: North Korea bans foreign tour...,Nearly 300 people have been infected in the co...,https://www.bbc.co.uk/news/world-asia-51194651,https://ichef.bbci.co.uk/news/1024/branded_new...,2020-01-21T19:33:22Z,Image copyrightGetty ImagesImage caption\r\n T...


,status,totalResults,source,author,title,description,url,urlToImage,publishedAt,content
0,ok,141,"{'id': 'the-verge', 'name': 'The Verge'}",Nicole Wetsman,If you’re feeling nervous about the coronaviru...,Reduce anxiety about the new coronavirus by pu...,https://www.theverge.com/2020/1/24/21080628/co...,https://cdn.vox-cdn.com/thumbor/aHRj-wE4dlipnM...,2020-01-24T21:50:50Z,Take a break from reading the internet\r\nPhot...


In [465]:
#extracting the descriptions of articles into array
descrip_reliable = pd.Series(news_reliable.description.values)
descrip_unreliable = pd.Series(news_unreliable.description.values)

In [466]:
# remove null entries
descrip_reliable = descrip_reliable.dropna()
descrip_unreliable = descrip_unreliable.dropna()

In [467]:
# Create the lists of the words that used for feature extraction
positive_list = list()
negative_list = list()

# Gather the positive and negative word lists from the text file
with open('./word_sentiment/positive-words.txt', encoding='ANSI') as p:
    for line in p:
        val = line.split()
        positive_list.append(val)

with open('./word_sentiment/negative-words.txt', encoding='ANSI') as n:
    for line in n:
        val = line.split()
        negative_list.append(val)

subjective_list = list(['am ', 'are ', 'is ', 'was ', 'were ', 'be ', 'been '])
causation_list = list(['led to ', 'because ', 'cause ', 'reason ', 'explanation ', 'so '])

In [468]:
#################

# Not required for right now, but may used in later for more efficient tockenization and feature extraction
# vectorizer = CountVectorizer()
# train_count = vectorizer.fit_transform(data["statement"].values)

# Check for vectorizer
# print(train_count.shape)
# print(vectorizer.vocabulary_)
# print(vectorizer.get_feature_names())

# Input: String ('str')
# Description: Tockenize and tag input with nltk universal tag
# Return: tags of each tockens - String ('tagged')
# Tockenize and tag input String with nltk universal tag and return the tags for each words in the String

def tagging_univ(str):
    text = nltk.word_tokenize(str)
    tagged = nltk.pos_tag(text, tagset='universal')
    return tagged

# Input: String('str')
# Description: Tockenize and tag input with nltk non-universal tag
# Return: tags of each tockens - String ('tagged')

def tagging_nuniv(str):
    text = nltk.word_tokenize(str)
    tagged = nltk.pos_tag(text)
    return tagged

# Input: String('str')
# Description: Count the number of characters in input
# Return: Character count - int ('count')

def count_char(str):
    no_space = str.replace(" ", "")
    count = len(no_space)
    return count

# Input: String('str')
# Description: Count the number of words in input
# Return: Word count - int ('count')

def count_word(str):
    count = len(str.split())
    return count

# Input: String ('tags'), int(verb_count)
# Description: Count the number of verbs in input
# Return: Verb count - int ('verb_count')

def count_verb(tags, verb_count):
    if (tags == 'VERB'):
        verb_count += 1
    return verb_count

# Input: String('tags'), int('noun_count')
# Description: Count the number of nouns in input
# Return: Noun count - int ('noun_count')

def count_noun(tags, noun_count):
    if (tags == 'NOUN'):
        noun_count += 1
    return noun_count

# Input: String ('str')
# Description: Count the number of sentences by counting number of period(.)
# Return: Sentence count - int ('sentence')

def count_sent(str):
    sentence = len(str.split('.'))
    return sentence

# Input: String ('states')
# Description: Count the number of characters in each word in input and average the number of characters per word
# Return: Average number of characters: float ('avg')

def count_char_per_word(states):
    word = []
    word.append(states.split())
    char_per_word = list()
    for elements in word:
        for char in elements:
            c_in_w_count = len(char)
            char_per_word.append(c_in_w_count)
    # char_per_word_list.append()
    avg = sum(char_per_word) / len(char_per_word)
    char_per_word.clear()
    return avg

# Input: String ('tagged'), list ('list'), int ('int')
# Description: Count the number of common words between input String and input list
# Return: Count of common words - int ('int')
# Count and return the number of common words between input String ('tagged') and input list

def check_common(tagged, list, int):
    for elements in list:
        if tagged in elements:
            int += 1
            return int
    return int

# Input: list ('subjective_list'), int ('verb_count'), String ('states')
# Description: Count the number of subjective words and avarage it from number of verbs in input
# Return: Average of subjective words - float ('percent_sub')

def count_sub(subjective_list, verb_count, states):
    percent_sub = 0
    for ele in subjective_list:
        sub_count = states.count(ele)
    if (verb_count > 0):
        percent_sub = sub_count / verb_count * 100
    return percent_sub

# Input: String ('word'), String ('tag'), list ('subjective_list'), int ('sent_count')
# Description: Count the number of passive voice and avarge it from number of sentence (sent_count)
# Return: Average of passive voiced sentences - float ('result')

def count_passive(word, tag, subjective_list, sent_count):
    percent_sub = 0
    counter = 0
    for ele in subjective_list:
        if (word.count(ele) > 0 and tag == "VBN"):
            counter += 1
    result = counter / sent_count * 100
    return result

# Input: String ('states')
# Description: Count the words that introduced only once in input
# Return: Count of unique words - int ('unique_count')

def count_unique(states):
    words = states.split(' ')
    c = Counter(words)
    unique = [w for w in words if c[w] == 1]
    unique_counter = len(unique)
    return unique_counter

# Input: String ('tag'), int ('typo_count')
# Description: Count the number of words with nltk universal tag is "X" (misspelled)
# Return: Count of misspelled words - int ('typo_count')

def count_typo(tag, typo_count):
    if (tag == "X"):
        typo_count += 1
    return typo_count

# Input: list ('causation_list'), String ('states')
# Description: Count the number of common words in list input and String input
# Return: Count of common words - int ('cause_count')

def count_cause(causation_list, states):
    for ele in causation_list:
        cause_count = states.count(ele)
    return cause_count

In [469]:
# Input: panda Series ('data'), String ('save_file_name')
# Description: First, this function creates lists for each features and extract the features using statements in dataset and
#              above functions. Next, it creates a large pandas Series that consist of news contents, labels and extracted
#              features. Finally, it save the final pandas Series in a file to make easier to examine the result (do not need
#              to rerun the program or change the code to check raw data)
# Return: Pandas Series consist of news contents, labels and extracted features count - pandas Series ('new')

def feature_extract(data, save_file_name):
    # define the news contents and labels from the dataset
    state = data

    # create lists for storing the counters
    char_count_list = list()
    word_count_list = list()
    verb_count_list = list()
    noun_count_list = list()
    sent_count_list = list()
    words_per_sent_list = list()
    char_per_word_list = list()
    quest_count_list = list()
    sub_count_list = list()
    pass_count_list = list()
    pos_count_list = list()
    neg_count_list = list()
    unique_count_list = list()
    typo_count_list = list()
    cause_count_list = list()
    word = list()

    #print(type(state))
    # loop for checking each new contents in dataset
    for states in state:

        #print(states)
        # reset the counters for each news contents
        w_in_s_count = 0
        c_in_w_count = 0
        verb_count = 0
        noun_count = 0
        pos_count = 0
        neg_count = 0
        percent_pos = 0
        percent_neg = 0
        unique_count = 0
        sent_counts = 0
        typo_count = 0

        # Tockenization and tagging with nltk universal and non-universal tag systems (tagged = universal, tagged_nu = non-universal)
        tagged = tagging_univ(states)
        tagged_nu = tagging_nuniv(states)

        # Check the tags of each news contents
        # print(tagged)
        # print(tagged_nu)

        # Extract the features and append the results in the list. Commented lines with print() functions are for testing

        # 1. Number of Characters
        char_count = count_char(states)
        char_count_list.append(char_count)

        # 2. Nubmer of Words
        word_count = count_word(states)
        word_count_list.append(word_count)

        # 3. Number of Verbs
        for tag in tagged:
            verb_count = count_verb(tag[1], verb_count)
        verb_count_list.append(verb_count)

        # 4. Number of Nouns
        for tag in tagged:
            noun_count = count_noun(tag[1], noun_count)
        noun_count_list.append(noun_count)
        # print(noun_count)
        
        # 5. Number of Sentence
        sent_count = count_sent(states)
        sent_count_list.append(sent_count)

        # 6. Average number of words per sentence
        sent = [len(l.split()) for l in re.split(r'[?!.]', states) if l.strip()]
        w_in_s_count = (sum(sent) / len(sent))
        words_per_sent_list.append(w_in_s_count)

        # 7. Average number of characters per word
        c_in_w_count = count_char_per_word(states)
        char_per_word_list.append(c_in_w_count)

        # 8. Number of question marks
        quest_count = states.count("?")
        quest_count_list.append(quest_count)

        # 9. Percentage of subjective verbs - am/are/is/etc
        percent_sub = count_sub(subjective_list, verb_count, states)
        sub_count_list.append(percent_sub)

        # 10. Percentage of passive voice - am/are/is && past participate
        for tag in tagged_nu:
            passive_percent = count_passive(tag[0], tag[1], subjective_list, sent_count)
        pass_count_list.append(passive_percent)

        # 11. Percentage of positive words
        for tag in tagged:
            pos_count = check_common(tag[0], positive_list, pos_count)
            percent_pos = pos_count / word_count * 100
        pos_count_list.append(percent_pos)

        # 12. Percentage of negative words
        for tag in tagged:
            neg_count = check_common(tag[0], negative_list, neg_count)
            percent_neg = neg_count / word_count * 100
        neg_count_list.append(percent_neg)
        
        # 13. Lexical diversity: unique words or terms
        unique_count = count_unique(states)
        unique_count_list.append(unique_count)

        # 14. Typographical error ratio: misspelled words
        for tag in tagged:
            typo_count = count_typo(tag[1], typo_count)
        typo_count_list.append(typo_count)
        
        # 15. Causation terms
        cause_count = count_cause(causation_list, states)
        cause_count_list.append(cause_count)

    # Convert lists into pandas Series
    statement = pd.Series(state)
    first = pd.Series(char_count_list)
    second = pd.Series(word_count_list)
    third = pd.Series(verb_count_list)
    fourth = pd.Series(noun_count_list)
    fifth = pd.Series(sent_count_list)
    sixth = pd.Series(words_per_sent_list)
    seventh = pd.Series(char_per_word_list)
    eighth = pd.Series(quest_count_list)
    nineth = pd.Series(sub_count_list)
    tenth = pd.Series(pass_count_list)
    eleventh = pd.Series(pos_count_list)
    twelveth = pd.Series(neg_count_list)
    thirteenth = pd.Series(unique_count_list)
    fourteenth = pd.Series(typo_count_list)
    fifteenth = pd.Series(cause_count_list)

    # Concatenate all feature Series, news contents and labels into one pandas Series
    new = pd.concat([statement, first, second, third, fourth, fifth, sixth, seventh, eighth, nineth, tenth,
                     eleventh, twelveth, thirteenth, fourteenth, fifteenth], axis=1)

    # Test for the final Series
    # print(new.head(3))

    # Name the header of the data
    new_header = ['Statement', '# of Characters', '# of Words', '# of Verbs', '# of Noun', '# of Sentence',
                  'Average # of Words per Sentence', 'Average # of Characters per Words', '# of Question Marks',
                  '% of Subjective Verbs', '% of Passive Voice', '% of Positive Words', '% of Negative Words',
                  '# of Unique Wrods/Terms', '# of Misspelled Words', '# of Causation Terms']

    # Save the data in a file

    #with open(save_file_name, 'w') as f:
    #    writer = csv.DictWriter(f, fieldnames=new_header)
    #    writer.writeheader()

    new.to_csv(save_file_name, header=new_header)
    return new

In [470]:
########################################################################################################################
# Feature Extraction
feature_extract(descrip_reliable, './reliable_FE.csv')
feature_extract(descrip_unreliable, './unreliable_FE.csv')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Nearly 300 people have been infected in the co...,85.0,15.0,4.0,5.0,2.0,15.0,5.666667,0.0,25.0,0.0,0.000000,13.333333,13.0,0.0,0.0
1,"A BBC team travels into Hubei province, where ...",72.0,13.0,1.0,6.0,2.0,13.0,5.538462,0.0,0.0,0.0,0.000000,7.692308,13.0,0.0,0.0
2,"Australian scientists say it is a ""significant...",89.0,15.0,3.0,4.0,2.0,15.0,5.933333,0.0,0.0,0.0,13.333333,0.000000,15.0,0.0,0.0
3,The new coronavirus is said to be infectious d...,87.0,17.0,6.0,3.0,2.0,17.0,5.117647,0.0,0.0,0.0,0.000000,0.000000,14.0,0.0,0.0
4,Jose Ramirez's unified WBC and WBO light-welte...,127.0,21.0,2.0,11.0,2.0,21.0,6.047619,0.0,0.0,0.0,0.000000,4.761905,21.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,The deadly coronavirus has sickened hundreds o...,90.0,16.0,1.0,8.0,2.0,16.0,5.625000,0.0,0.0,0.0,0.000000,6.250000,14.0,0.0,0.0
96,Social media network has blocked only some use...,100.0,20.0,3.0,7.0,2.0,20.0,5.000000,0.0,0.0,0.0,0.000000,5.000000,16.0,0.0,0.0
97,The coronavirus outbreak started in central Ch...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,CBS News correspondent Ramy Inocencio has the ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Reduce anxiety about the new coronavirus by pu...,124,24,3,10,2,24.000000,5.166667,0,0.0,0.0,0.000000,12.500000,21,0,0
1,Google is trying to foreground sourcing and UR...,214,47,12,15,3,15.666667,4.553191,0,0.0,0.0,4.255319,2.127660,35,0,0
2,Mysterious viral pneumonia outbreak in China l...,217,44,6,21,1,22.000000,4.931818,1,0.0,0.0,0.000000,11.363636,28,0,0
3,"Developers of the popular game, in which playe...",134,29,5,11,2,15.000000,4.620690,0,0.0,0.0,3.448276,6.896552,22,0,0
4,The World Health Organization commended the de...,195,39,7,13,3,19.500000,5.000000,0,0.0,0.0,0.000000,5.128205,24,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Screenings for passengers arriving from Wuhan,...",102,19,3,11,2,19.000000,5.368421,0,0.0,0.0,0.000000,0.000000,19,0,0
96,The Centers for Disease Control and Prevention...,223,38,4,22,3,13.000000,5.868421,0,0.0,0.0,0.000000,0.000000,30,0,0
97,The man developed a cough while still in China...,97,20,3,5,2,20.000000,4.850000,0,0.0,0.0,5.000000,5.000000,16,0,0
98,The U.S. government is sending a charter fligh...,216,45,7,15,3,15.333333,4.800000,0,0.0,0.0,0.000000,6.666667,33,0,0
